### **6.6 - Dealing with resource consuming tasks with Pools**

Para ilustrar una de mis características favoritas en Airflow, permítanme mostrarles un ejemplo concreto. Imaginemos que tenemos el siguiente data pipeline con una tarea encargada de extraer los datos. A continuación, tres tareas encargadas de entrenar los modelos de machine learning 1, 2 y tres. Y, por último, pero no menos importante, dos tareas encargadas de probar esos modelos. Ahora bien, puede que las tareas "Train ML" consuman mucho, porque se encargan de entrenar los modelos de machine learning, consumirán mucha CPU o mucha memoria. Así que una cosa que usted podría querer es ejecutar una tarea a la vez, sólo para este conjunto específico de tareas, pero, todavía quieres ser capaz de ejecutar las tareas "Test ML" al mismo tiempo. Entonces, ¿cómo se puede decir que, sólo para estas tareas específicas (Train ML tasks), quiero ejecutar una tarea a la vez, pero, para este conjunto de tareas (Test ML tasks), quiero ser capaz de ejecutarlas al mismo tiempo. ¿Cómo se puede hacer esto? Utilizando "Pools".

<center><img src="https://i.postimg.cc/QtFx2jcb/a1381.png"></center>

El concepto de pool es extremadamente simple, tienes un pool con un número de worker slots y cada vez que una tarea se está ejecutando entonces esa tarea toma un worker slot hasta que la tarea se completa y entonces ese slot se libera. Así es como funciona, es tan simple como eso. Y por defecto, todas tus tareas se ejecutan dentro del mismo pool llamado "default_pool". ¿Cómo se puede comprobar esto? En Airflow UI, si vas a Admin > Pools

<center><img src="https://i.postimg.cc/fTmkWWpq/a1382.png"></center>

Puedes ver el "default_pool", "default_pool" con 128 slots, lo que significa que puedes ejecutar como máximo 128 slots al mismo tiempo en toda tu instancia de airflow. Obviamente puedes modificar ese número, pero, ¿qué número deberías poner ahí? Eso depende de tus recursos y de tus casos de uso, pero tienes que saber que por defecto, siempre que una tarea se está ejecutando, si no has especificado ningún pool entonces tu tarea se está ejecutando dentro de ese pool. Y como puedes ver tienes "Running slots" y "Queued slots". Así que básicamente, si el pool se llena, la siguiente tarea a ejecutar se pondrá en cola y verás que el número de "Queued slots" aumenta y tan pronto como la tarea se completa, el worker slot se libera y entonces una de las tareas en cola tomará ese worker slot para ejecutarse. Así es como funciona. ¿Y ahora por qué necesitas pools? Bueno, porque con los pools puedes definir la "concurrencia", el número de tareas que se ejecutan al mismo tiempo para un conjunto específico de tareas. 

<center><img src="https://i.postimg.cc/xjkc73R2/a1383.png"></center>

Permítanme mostrarles esto. Si haces clic en tu DAG y vas a la Graph View

<center><img src="https://i.postimg.cc/Vs3dc0Gb/a1384.png"></center>
<center><img src="https://i.postimg.cc/XqFNvxHB/a1385.png"></center>

Digamos que esas tareas, "extract_partner_snowflake", "extract_partner_netflix" y "extract_partner_astronomer", consumen muchos recursos. Así que una cosa que puedes querer es ejecutar una tarea a la vez, pero sólo para este conjunto de tareas (sólo para estas tres tareas). 

<center><img src="https://i.postimg.cc/qBCvQMJB/a1386.png"></center>

Mientras que, aún quieres ejecutar tantas tareas como puedas en paralelo, que son 16 por defecto para la otra tarea de tu DAG. ¿Cómo hacerlo? Bien, aquí es donde necesitas crear un "pool". Creas un nuevo pool con un worker slot, y asignas ese pool a todas esas tareas "extract_partner_snowflake", "extract_partner_netflix" y "extract_partner_astronomer". Así que el primer paso es crear ese pool. Vuelve a Admin > Pools, haz clic en añadir un nuevo registro

<center><img src="https://i.postimg.cc/4djNZj5T/a1387.png"></center>

y llamemos a ese pool "partner_pool" con un worker slot y una descripción "Pool for extract tasks". Haz clic en guardar:

<center><img src="https://i.postimg.cc/NfqfnqF6/a1388.png"></center>

Ya tienes tu pool "partner_pool" con un slot, lo que significa que siempre tendrás una tarea a la vez ejecutándose dentro de ese pool.

<center><img src="https://i.postimg.cc/X7f7SHBT/a1389.png"></center>

Así que ahora, el siguiente paso es volver a tu data pipeline y echar un vistazo a la tarea "extract", donde como argumento quieres definir:

<center><img src="https://i.postimg.cc/nhnVH4vM/a1390.png"></center>
<center><img src="https://i.postimg.cc/Mpt6XnD5/a1391.png"></center>

Haciendo esto, en lugar de ejecutar "extract" dentro del pool "default_pool", vas a ejecutar "extract" dentro del pool "partner_pool", que tiene un worker slot y así tendrás una tarea ejecutándose a la vez. Sólo para comprobar si funciona, necesitas comentar algunas líneas aquí, así que primero comenta "choosing_partner_based_on_day" ya que quieres ejecutar todas las tareas extract. Lo mismo para "stop", esta línea y no deseas utilizar "choosing_partner_based_on_day", así que puedes eliminarla.

<center><img src="https://i.postimg.cc/3wRKG3fT/a1392.png"></center>
<center><img src="https://i.postimg.cc/LsbmBGGZ/a1393.png"></center>

Entonces la última cosa que puedes hacer es usar "sleep" para que realmente puedas ver si tus tareas se ejecutan de una en una, de lo contrario, será demasiado rápido. Así que primero importa time

<center><img src="https://i.postimg.cc/NM2BNpwC/a1394.png"></center>

y luego en la tarea "extract" escribe:

<center><img src="https://i.postimg.cc/prDxvrrD/a1395.png"></center>

Guarda el archivo y vuelve a la interfaz de Airflow y deberías obtener el siguiente data pipeline. Vamos a activar el toggle del DAG para ver qué pasa.

<center><img src="https://i.postimg.cc/XqM3tdjj/a1396.png"></center>

Actualiza la página, ok, "start" se ha completado y ahora puedes ver que "extract_partner_snowflake" se está ejecutando mientras que los otros están "en cola". 

<center><img src="https://i.postimg.cc/pXHRZfck/a1397.png"></center>

Ahora "extract_partner_astronomer"

<center><img src="https://i.postimg.cc/t4Ry0Xjx/a1398.png"></center>

y finalmente "extract_partner_netflix". 

<center><img src="https://i.postimg.cc/qvmr49JZ/a1399.png"></center>

Así que realmente se puede ver que esas tareas se ejecutan una tarea a la vez, porque se utiliza un pool "partner_pool". Así que esa es la belleza de los pools, no dudes en usarlos, si necesitas modificar el número de tareas a ejecutar en paralelo para un conjunto específico de tareas. **`Adicionalmente, notas importantes sobre los pools. La primera es que hay un argumento que puedes usar en cualquier operador que se llama "pool_slots" con el valor por defecto fijado en "1", y este argumento define el número de worker slots que toma una tarea dada.`**

<center><img src="https://i.postimg.cc/zfVrB03D/a1400.png"></center>

Así que básicamente, si dices que "extract" toma tres worker slots, entonces esa tarea tomará tres worker slots en tu pool tan pronto como se dispare. 

<center><img src="https://i.postimg.cc/43qThVh6/a1401.png"></center>

Además, tienes que saber que si utilizas el Operador SubDag, que si recuerdas espera el "task_id" y el "subdag" con una función. Si configuras el pool como "partner_pool", este pool no será respetado por las tareas de tu SubDag. Es realmente importante recordar esto, si se establece un pool, en el Operador SubDag, este pool no será utilizado por las tareas dentro de su SubDag. De hecho, sólo el Operador SubDag respetará ese pool. Así que, si necesitas modificar la "concurrencia" de tu tarea dentro de tu SubDag, entonces necesitas definir el pool para cada tarea de tu SubDag.

<center><img src="https://i.postimg.cc/hjmR8v9H/a1402.png"></center>

Por último, hay una manera de definir las prioridades de las tareas en tu DAG y para ello necesitas aprovechar el pool. Pero primero, veamos en el siguiente video cómo puedes definir esas prioridades.